# PROYECTO INDIVIDUAL Nº2

***Data Analitycs***

Previo se realizó en análisis de los siguientes puntos:

- Consulta de la documentación API de la página CoinGecko.

- Análisis de los endpoints para la obtención de la data.

- Elección del endpoint /coins/{id}/market_chart/range

**1 - Importación de librerías**

In [1]:
#Importación de la librerias
import datetime
import requests
import pandas as pd
import csv
import numpy as np
from datetime import timedelta



**2 - Definición de funciones**

2.a - Funciones para transformación de formatos de fechas

In [2]:
#Definir la función para transformar de string a formato unix timestamp

def timestamp_unix(fecha):
    fecha_dt = datetime.datetime.strptime(fecha, '%d-%m-%Y')
    timestamp_unix = int(fecha_dt.timestamp())

    return timestamp_unix

In [3]:
# Definir la función para convertir el unix timestamp en una fecha formateada
def conv_fecha(timestamp_millis):
    timestamp_seconds = timestamp_millis / 1000
    fecha = datetime.datetime.fromtimestamp(timestamp_seconds).date()
    fecha_formateada = fecha.strftime('%d-%m-%Y')
    return fecha_formateada

2.b - Función para consulta de enpoint

In [4]:
#Definir la función para la consulta del endpoint
def get_api(url,unix_from,unix_to):

    params = {
        "vs_currency": "usd",
        "from": unix_from,  # Timestamp de inicio
        "to": unix_to,    # Timestamp de fin
        "precision": "full"
    }

    response = requests.get(url, params=params)     # Realizar la solicitud GET a la API

    if response.status_code == 200:                 # Verificar si la solicitud fue exitosa
        
        data = response.json()                      # Obtener los datos JSON de la respuesta

        return data
        
    else:
        print("Error al realizar la solicitud:", response.status_code)

2.c - Función para creación de data frames

In [5]:
#Definir función para creación de data frames
def creardf (data0):
    
    df_prices = pd.DataFrame(data0["prices"], columns=["timestamp", "price"])                       # Convertir los datos en un DataFrame
    df_market_caps = pd.DataFrame(data0["market_caps"], columns=["timestamp", "market_cap"])
    df_total_volumes = pd.DataFrame(data0["total_volumes"], columns=["timestamp", "total_volume"])
 
    df = df_prices.merge(df_market_caps, on="timestamp").merge(df_total_volumes, on="timestamp")    # Fusionar los DataFrames en uno 
                                                                                                    # solo usando el campo 'timestamp' como clave
    return df

    del df_prices
    del df_market_caps
    del df_total_volumes

2.d Función Media Móvil Exponencial (EMA)

In [6]:
#Definir función Media Móvil Exponencial (EMA)
def ema(df,N):

    K = 2 / (N + 1)                                             # Calcular el factor de suavizado K

    return df['price'].ewm(span=N, adjust=False).mean()         # Calcular la EMA utilizando la función ewm().mean()

2.e Función Media Móvil de Convergencia / Divergencia (MACD)

In [7]:
#Definir función Media Móvil de Convergencia / Divergencia (MACD)
def macd (df1):

    df1['MACD'] = df1['EMA12'] - df1['EMA26']                          # Calcular el MACD
   
    df1['signal_line'] = df1['MACD'].rolling(window=9).mean()         # Calcular la señal del MACD (9EMA del MACD)

    return df1['MACD'],df1['signal_line']

2.f Función para cálcular el Índice de Fuerza Relativa (RSI)

In [8]:
#Definir función para cálcular el Índice de Fuerza Relativa (RSI)
def rsi(df2):
    
    df2['price_change'] = df2['price'].diff()   # Calcular el cambio de precio (diferencia entre cierres)

    
    df2['gain'] = np.where(df2['price_change'] > 0, df2['price_change'], 0) # Calcular ganancias (positivas) y pérdidas (negativas)
    df2['loss'] = np.where(df2['price_change'] < 0, -df2['price_change'], 0)

    
    window = 14  # Período recomendado para RSI
    df2['ema_gain'] = df2['gain'].ewm(span=window, adjust=False).mean() # Calcular promedios móviles exponenciales de ganancias y pérdidas
    df2['ema_loss'] = df2['loss'].ewm(span=window, adjust=False).mean()

    
    df2['rs'] = df2['ema_gain'] / df2['ema_loss']       # Calcular el RSI
    df2['rsi'] = 100 - (100 / (1 + df2['rs']))

    return df2['rsi']

2.g Función para obtener los ROI

In [9]:
def valorROI(df2,plazo):
    # Obtener el valor máximo de la columna timestamp
    filtro1 = df2['date'].iloc[-1]

    # Obtener la fecha actual
    fecha_1 = datetime.datetime.strptime(filtro1, "%d-%m-%Y")

    # Calcular la fecha de un año atrás
    fecha_2 = fecha_1 - timedelta(days=plazo)

    # Convertir la fecha al formato deseado
    filtro2 = fecha_2.strftime("%d-%m-%Y")

    # Precios de compra y venta en USD
    precio_compra = df2[df2['date'] == filtro2]['price'].iloc[0]
    precio_venta = df2[df2['date'] == filtro1]['price'].iloc[0]

    # Calcular el ROI
    ganancia = precio_venta - precio_compra
    costo = precio_compra

    roi = (ganancia / costo) * 100

    return roi

**3 - Definición de variables fecha y otras**

In [10]:
fecha_actual = datetime.date.today()                            #Fecha actual

fecha_inicio = fecha_actual - datetime.timedelta(days=(365))  #Fecha inicial de la consulta de datos

fecha_from = str(fecha_inicio.strftime('%d-%m-%Y'))             #Conversión de fechas a string

fecha_to = str(fecha_actual.strftime('%d-%m-%Y'))

unix_from = timestamp_unix(fecha_from)                          #Conversión de fechas a unix timestamp
unix_to = timestamp_unix(fecha_to)

new_endpoint = "/market_chart/range"

**4 - Obtención de la información de la API y creación de los data frames**

4.a - Solana SOL

In [11]:
base_url = "https://api.coingecko.com/api/v3/coins/solana"  # Definir la URL de la API con los parámetros de consulta

url = base_url + new_endpoint       # Combinar las partes para formar la nueva URL


data0 = get_api(url,unix_from,unix_to)

In [12]:
df_solana = creardf(data0)  #Creación del data frame con la función

In [13]:
df_solana.head()

,timestamp,price,market_cap,total_volume
0,1660608000000,43.902581,1.532266e+10,1.018587e+09
1,1660694400000,43.252084,1.506556e+10,1.075538e+09
2,1660780800000,40.556286,1.416116e+10,1.249878e+09
3,1660867200000,39.618895,1.386326e+10,8.249718e+08
4,1660953600000,35.989795,1.253047e+10,1.786764e+09


4.b Bitcoin BTC

In [14]:
base_url = "https://api.coingecko.com/api/v3/coins/bitcoin"  # Definir la URL de la API con los parámetros de consulta

url = base_url + new_endpoint       # Combinar las partes para formar la nueva URL

data0 = get_api(url,unix_from,unix_to)

In [15]:
df_bitcoin = creardf(data0)  #Creación del data frame con la función

In [16]:
df_bitcoin.head()

,timestamp,price,market_cap,total_volume
0,1660608000000,24179.014652,4.624372e+11,3.340481e+10
1,1660694400000,23913.333726,4.567910e+11,2.497130e+10
2,1660780800000,23359.418837,4.470158e+11,2.956740e+10
3,1660867200000,23248.384228,4.448460e+11,2.022821e+10
4,1660953600000,20945.833966,4.008104e+11,3.514241e+10


4.c Tether USDT

In [17]:
base_url = "https://api.coingecko.com/api/v3/coins/tether"  # Definir la URL de la API con los parámetros de consulta

url = base_url + new_endpoint       # Combinar las partes para formar la nueva URL

data0 = get_api(url,unix_from,unix_to)

In [18]:
df_tether = creardf(data0)  #Creación del data frame con la función

In [19]:
df_tether.head()

,timestamp,price,market_cap,total_volume
0,1660608000000,1.001842,6.765299e+10,5.762541e+10
1,1660694400000,1.000989,6.762839e+10,4.820835e+10
2,1660780800000,1.000875,6.766131e+10,5.435838e+10
3,1660867200000,1.001241,6.759940e+10,4.209014e+10
4,1660953600000,1.003335,6.761379e+10,6.330288e+10


4.d USD Coin USDC

In [20]:
base_url = "https://api.coingecko.com/api/v3/coins/usd-coin"  # Definir la URL de la API con los parámetros de consulta

url = base_url + new_endpoint       # Combinar las partes para formar la nueva URL

data0 = get_api(url,unix_from,unix_to)

In [21]:
df_usd_coin = creardf(data0)  #Creación del data frame con la función

In [22]:
df_usd_coin.head()

,timestamp,price,market_cap,total_volume
0,1660608000000,1.001911,5.358637e+10,7.323677e+09
1,1660694400000,1.001137,5.356327e+10,6.057253e+09
2,1660780800000,1.000239,5.319273e+10,6.330005e+09
3,1660867200000,1.000279,5.288017e+10,5.127734e+09
4,1660953600000,1.002019,5.252307e+10,7.381892e+09


4.e Ethereum ETH

In [23]:
base_url = "https://api.coingecko.com/api/v3/coins/ethereum"  # Definir la URL de la API con los parámetros de consulta

url = base_url + new_endpoint       # Combinar las partes para formar la nueva URL

data0 = get_api(url,unix_from,unix_to)

In [24]:
df_ethereum = creardf(data0)  #Creación del data frame con la función

In [25]:
df_ethereum.head()

,timestamp,price,market_cap,total_volume
0,1660608000000,1908.277642,2.294404e+11,1.831087e+10
1,1660694400000,1880.600101,2.255816e+11,1.424023e+10
2,1660780800000,1834.826453,2.206659e+11,1.842755e+10
3,1660867200000,1850.111291,2.226524e+11,1.336378e+10
4,1660953600000,1618.532504,1.948177e+11,2.268736e+10


4.f Cardano ADA

In [26]:
base_url = "https://api.coingecko.com/api/v3/coins/cardano"  # Definir la URL de la API con los parámetros de consulta

url = base_url + new_endpoint       # Combinar las partes para formar la nueva URL

data0 = get_api(url,unix_from,unix_to)

In [27]:
df_cardano = creardf(data0)  #Creación del data frame con la función

In [28]:
df_cardano.head()

,timestamp,price,market_cap,total_volume
0,1660608000000,0.552459,1.869741e+10,8.236799e+08
1,1660694400000,0.559785,1.892784e+10,6.807133e+08
2,1660780800000,0.539302,1.821255e+10,9.965319e+08
3,1660867200000,0.514487,1.741240e+10,5.741756e+08
4,1660953600000,0.452547,1.526462e+10,1.131870e+09


4.g Avalanche AVAX

In [29]:
base_url = "https://api.coingecko.com/api/v3/coins/avalanche-2"  # Definir la URL de la API con los parámetros de consulta

url = base_url + new_endpoint       # Combinar las partes para formar la nueva URL

data0 = get_api(url,unix_from,unix_to)

In [30]:
df_avalanche = creardf(data0)  #Creación del data frame con la función

In [31]:
df_avalanche.head()

,timestamp,price,market_cap,total_volume
0,1660608000000,27.878861,7.904216e+09,4.390438e+08
1,1660694400000,27.251164,7.763571e+09,3.659226e+08
2,1660780800000,25.530010,7.283666e+09,4.260619e+08
3,1660867200000,24.299012,6.932924e+09,3.173880e+08
4,1660953600000,22.535468,6.421005e+09,6.394015e+08


4.h BNB BNB

In [32]:
base_url = "https://api.coingecko.com/api/v3/coins/binancecoin"  # Definir la URL de la API con los parámetros de consulta

url = base_url + new_endpoint       # Combinar las partes para formar la nueva URL

data0 = get_api(url,unix_from,unix_to)

In [33]:
df_bnb = creardf(data0)  #Creación del data frame con la función

In [34]:
df_bnb .head()

,timestamp,price,market_cap,total_volume
0,1660608000000,319.974835,5.225906e+10,1.029621e+09
1,1660694400000,316.698400,5.164471e+10,1.009809e+09
2,1660780800000,306.639287,5.010061e+10,1.199734e+09
3,1660867200000,298.600700,4.882570e+10,1.091856e+09
4,1660953600000,279.977814,4.564295e+10,1.657993e+09


4.i Dogecoin DOGE

In [35]:
base_url = "https://api.coingecko.com/api/v3/coins/dogecoin"  # Definir la URL de la API con los parámetros de consulta

url = base_url + new_endpoint       # Combinar las partes para formar la nueva URL

data0 = get_api(url,unix_from,unix_to)

In [36]:
df_dogecoin = creardf(data0)  #Creación del data frame con la función

In [37]:
df_dogecoin.head()

,timestamp,price,market_cap,total_volume
0,1660608000000,0.076939,1.022052e+10,8.810870e+08
1,1660694400000,0.087191,1.161382e+10,1.907993e+09
2,1660780800000,0.080599,1.070143e+10,9.878966e+08
3,1660867200000,0.075108,9.988655e+09,5.991804e+08
4,1660953600000,0.068219,9.037215e+09,1.015524e+09


4.j EOS EOS

In [38]:
base_url = "https://api.coingecko.com/api/v3/coins/eos"  # Definir la URL de la API con los parámetros de consulta

url = base_url + new_endpoint       # Combinar las partes para formar la nueva URL

data0 = get_api(url,unix_from,unix_to)

In [39]:
df_eos = creardf(data0)  #Creación del data frame con la función

In [40]:
df_eos.head()

,timestamp,price,market_cap,total_volume
0,1660608000000,1.275740,1.282014e+09,2.234985e+08
1,1660694400000,1.377330,1.378720e+09,3.101783e+08
2,1660780800000,1.483177,1.492323e+09,1.224827e+09
3,1660867200000,1.480666,1.493785e+09,5.873258e+08
4,1660953600000,1.276770,1.281015e+09,5.529318e+08


**5 - Proceso EDA**

5.a Creación y eliminación de columnas

In [41]:
df_solana['date'] = df_solana['timestamp'].apply(conv_fecha)    # Aplicar la función a la columna y crear una nueva columna

In [42]:
df_solana['nombre'] = 'Solana'                                  #Creación de columna con el nombre de la moneda

In [43]:
df_bitcoin['date'] = df_bitcoin['timestamp'].apply(conv_fecha)  # Aplicar la función a la columna y crear una nueva columna

In [44]:
df_bitcoin['nombre'] = 'Bitcoin'                                #Creación de columna con el nombre de la moneda

In [45]:
df_tether['date'] = df_tether['timestamp'].apply(conv_fecha)    # Aplicar la función a la columna y crear una nueva columna

In [46]:
df_tether['nombre'] = 'Tether'                                  #Creación de columna con el nombre de la moneda

In [47]:

df_usd_coin['date'] = df_usd_coin['timestamp'].apply(conv_fecha)    # Aplicar la función a la columna y crear una nueva columna

In [48]:
df_usd_coin['nombre'] = 'USD Coin'                                  #Creación de columna con el nombre de la moneda

In [49]:
df_ethereum['date'] = df_ethereum['timestamp'].apply(conv_fecha)    # Aplicar la función a la columna y crear una nueva columna

In [50]:
df_ethereum['nombre'] = 'Ethereum'                                  #Creación de columna con el nombre de la moneda

In [51]:
df_cardano['date'] = df_cardano['timestamp'].apply(conv_fecha)      # Aplicar la función a la columna y crear una nueva columna

In [52]:
df_cardano['nombre'] = 'Cardano'                                    #Creación de columna con el nombre de la moneda

In [53]:
df_avalanche['date'] = df_avalanche['timestamp'].apply(conv_fecha)  # Aplicar la función a la columna y crear una nueva columna

In [54]:
df_avalanche['nombre'] = 'Avalanche'                                #Creación de columna con el nombre de la moneda

In [55]:
df_bnb['date'] = df_bnb['timestamp'].apply(conv_fecha)      # Aplicar la función a la columna y crear una nueva columna

In [56]:
df_bnb['nombre'] = 'BNB'                                    #Creación de columna con el nombre de la moneda

In [57]:
df_dogecoin['date'] = df_dogecoin['timestamp'].apply(conv_fecha)    # Aplicar la función a la columna y crear una nueva columna

In [58]:
df_dogecoin['nombre'] = 'Dogecoin'                              #Creación de columna con el nombre de la moneda

In [59]:
df_eos['date'] = df_eos['timestamp'].apply(conv_fecha)          # Aplicar la función a la columna y crear una nueva columna

In [60]:
df_eos['nombre'] = 'EOS'                                        #Creación de columna con el nombre de la moneda

5.b Cálculo Media Móvil Exponencial (EMA)


EMA12

In [61]:
# Elegir el período de tiempo para la EMA
N = 12

In [62]:
dfaux = df_solana
df_solana['EMA12'] = ema(dfaux, N)             #Creación de la columna EMA12 con la función ema

In [63]:
dfaux = df_bitcoin
df_bitcoin['EMA12'] = ema(dfaux, N)             #Creación de la columna EMA12 con la función ema

In [64]:
dfaux = df_tether
df_tether['EMA12'] = ema(dfaux, N)             #Creación de la columna EMA12 con la función ema

In [65]:
dfaux = df_usd_coin
df_usd_coin['EMA12'] = ema(dfaux, N)             #Creación de la columna EMA12 con la función ema

In [66]:
dfaux = df_ethereum
df_ethereum['EMA12'] = ema(dfaux, N)             #Creación de la columna EMA12 con la función ema

In [67]:
dfaux = df_cardano
df_cardano['EMA12'] = ema(dfaux, N)             #Creación de la columna EMA12 con la función ema

In [68]:
dfaux = df_avalanche
df_avalanche['EMA12'] = ema(dfaux, N)             #Creación de la columna EMA12 con la función ema

In [69]:
dfaux = df_bnb
df_bnb['EMA12'] = ema(dfaux, N)                     #Creación de la columna EMA12 con la función ema

In [70]:
dfaux = df_dogecoin
df_dogecoin['EMA12'] = ema(dfaux, N)                     #Creación de la columna EMA12 con la función ema

In [71]:
dfaux = df_eos
df_eos['EMA12'] = ema(dfaux, N)                     #Creación de la columna EMA12 con la función ema

EMA26

In [72]:
# Elegir el período de tiempo para la EMA
N = 26

In [73]:
dfaux = df_solana
df_solana['EMA26'] = ema(dfaux, N)             #Creación de la columna EMA26 con la función ema

In [74]:
dfaux = df_bitcoin
df_bitcoin['EMA26'] = ema(dfaux, N)             #Creación de la columna EMA12 con la función ema

In [75]:
dfaux = df_tether
df_tether['EMA26'] = ema(dfaux, N)             #Creación de la columna EMA12 con la función ema

In [76]:
dfaux = df_usd_coin
df_usd_coin['EMA26'] = ema(dfaux, N)             #Creación de la columna EMA26 con la función ema

In [77]:
dfaux = df_ethereum
df_ethereum['EMA26'] = ema(dfaux, N)             #Creación de la columna EMA26 con la función ema

In [78]:
dfaux = df_cardano
df_cardano['EMA26'] = ema(dfaux, N)             #Creación de la columna EMA26 con la función ema

In [79]:
dfaux = df_avalanche
df_avalanche['EMA26'] = ema(dfaux, N)             #Creación de la columna EMA26 con la función ema

In [80]:
dfaux = df_bnb
df_bnb['EMA26'] = ema(dfaux, N)                     #Creación de la columna EMA26 con la función ema

In [81]:
dfaux = df_dogecoin
df_dogecoin['EMA26'] = ema(dfaux, N)                     #Creación de la columna EMA26 con la función ema

In [82]:
dfaux = df_eos
df_eos['EMA26'] = ema(dfaux, N)                     #Creación de la columna EMA26 con la función ema

EMA200

In [83]:
# Elegir el período de tiempo para la EMA
N = 200

In [84]:
dfaux = df_solana
df_solana['EMA200'] = ema(dfaux, N)             #Creación de la columna EMA200 con la función ema

In [85]:
dfaux = df_bitcoin
df_bitcoin['EMA200'] = ema(dfaux, N)             #Creación de la columna EMA12 con la función ema

In [86]:
dfaux = df_tether
df_tether['EMA200'] = ema(dfaux, N)             #Creación de la columna EMA200 con la función ema

In [87]:
dfaux = df_usd_coin
df_usd_coin['EMA200'] = ema(dfaux, N)             #Creación de la columna EMA200 con la función ema

In [88]:
dfaux = df_ethereum
df_ethereum['EMA200'] = ema(dfaux, N)             #Creación de la columna EMA200 con la función ema

In [89]:
dfaux = df_cardano
df_cardano['EMA200'] = ema(dfaux, N)             #Creación de la columna EMA200 con la función ema

In [90]:
dfaux = df_avalanche
df_avalanche['EMA200'] = ema(dfaux, N)             #Creación de la columna EMA200 con la función ema

In [91]:
dfaux = df_bnb
df_bnb['EMA200'] = ema(dfaux, N)                     #Creación de la columna EMA200 con la función ema

In [92]:
dfaux = df_dogecoin
df_dogecoin['EMA200'] = ema(dfaux, N)                     #Creación de la columna EMA200 con la función ema

In [93]:
dfaux = df_eos
df_eos['EMA200'] = ema(dfaux, N)                     #Creación de la columna EMA200 con la función ema

5.c Cálculo de la Media Móvil de Convergencia / Divergencia (MACD)

In [94]:
df1 = df_solana

macd_values, signal_line_values = macd(df1)     # Aplicar la función macd al DataFrame

df_solana['MACD'] = macd_values                    # Agregar los valores calculados de MACD y señal al DataFrame
df_solana['signal_line'] = signal_line_values

In [95]:
df1 = df_bitcoin

macd_values, signal_line_values = macd(df1)     # Aplicar la función macd al DataFrame

df_bitcoin['MACD'] = macd_values                    # Agregar los valores calculados de MACD y señal al DataFrame
df_bitcoin['signal_line'] = signal_line_values

In [96]:
df1 = df_tether

macd_values, signal_line_values = macd(df1)     # Aplicar la función macd al DataFrame

df_tether['MACD'] = macd_values                    # Agregar los valores calculados de MACD y señal al DataFrame
df_tether['signal_line'] = signal_line_values

In [97]:
df1 = df_usd_coin

macd_values, signal_line_values = macd(df1)     # Aplicar la función macd al DataFrame

df_usd_coin['MACD'] = macd_values                    # Agregar los valores calculados de MACD y señal al DataFrame
df_usd_coin['signal_line'] = signal_line_values

In [98]:
df1 = df_ethereum

macd_values, signal_line_values = macd(df1)     # Aplicar la función macd al DataFrame

df_ethereum['MACD'] = macd_values                    # Agregar los valores calculados de MACD y señal al DataFrame
df_ethereum['signal_line'] = signal_line_values

In [99]:
df1 = df_cardano

macd_values, signal_line_values = macd(df1)     # Aplicar la función macd al DataFrame

df_cardano['MACD'] = macd_values                    # Agregar los valores calculados de MACD y señal al DataFrame
df_cardano['signal_line'] = signal_line_values

In [100]:
df1 = df_avalanche

macd_values, signal_line_values = macd(df1)     # Aplicar la función macd al DataFrame

df_avalanche['MACD'] = macd_values                    # Agregar los valores calculados de MACD y señal al DataFrame
df_avalanche['signal_line'] = signal_line_values

In [101]:
df1 = df_bnb

macd_values, signal_line_values = macd(df1)     # Aplicar la función macd al DataFrame

df_bnb['MACD'] = macd_values                    # Agregar los valores calculados de MACD y señal al DataFrame
df_bnb['signal_line'] = signal_line_values

In [102]:
df1 = df_dogecoin

macd_values, signal_line_values = macd(df1)     # Aplicar la función macd al DataFrame

df_dogecoin['MACD'] = macd_values                    # Agregar los valores calculados de MACD y señal al DataFrame
df_dogecoin['signal_line'] = signal_line_values

In [103]:
df1 = df_eos

macd_values, signal_line_values = macd(df1)     # Aplicar la función macd al DataFrame

df_eos['MACD'] = macd_values                    # Agregar los valores calculados de MACD y señal al DataFrame
df_eos['signal_line'] = signal_line_values

5.d Índice de Fuerza Relativa (RSI) en criptomonedas

In [104]:
df2 = df_solana

df_solana['rsi'] = rsi(df2)                 #Crear columna cálculada del rsi

In [105]:
df2 = df_bitcoin

df_bitcoin['rsi'] = rsi(df2)                 #Crear columna cálculada del rsi

In [106]:
df2 = df_tether

df_tether['rsi'] = rsi(df2)                 #Crear columna cálculada del rsi

In [107]:
df2 = df_usd_coin

df_usd_coin['rsi'] = rsi(df2)                 #Crear columna cálculada del rsi

In [108]:
df2 = df_ethereum

df_ethereum['rsi'] = rsi(df2)                 #Crear columna cálculada del rsi

In [109]:
df2 = df_cardano

df_cardano['rsi'] = rsi(df2)                 #Crear columna cálculada del rsi

In [110]:
df2 = df_avalanche

df_avalanche['rsi'] = rsi(df2)                 #Crear columna cálculada del rsi

In [111]:
df2 = df_bnb

df_bnb['rsi'] = rsi(df2)                 #Crear columna cálculada del rsi

In [112]:
df2 = df_dogecoin

df_dogecoin['rsi'] = rsi(df2)                 #Crear columna cálculada del rsi

In [113]:
df2 = df_eos

df_eos['rsi'] = rsi(df2)                 #Crear columna cálculada del rsi

5.e Cálculo de los ROI

In [114]:
plazo1 = 30
plazo2 = 90
plazo3 = 180

In [115]:
df2 = df_solana

df_solana['ROI30'] = valorROI(df2,plazo1)        #Creación y cálculo de la columna ROI30
df_solana['ROI90'] = valorROI(df2,plazo2)        #Creación y cálculo de la columna ROI90
df_solana['ROI180'] = valorROI(df2,plazo3)        #Creación y cálculo de la columna ROI180

In [116]:
df2 = df_bitcoin

df_bitcoin['ROI30'] = valorROI(df2,plazo1)        #Creación y cálculo de la columna ROI30
df_bitcoin['ROI90'] = valorROI(df2,plazo2)        #Creación y cálculo de la columna ROI90
df_bitcoin['ROI180'] = valorROI(df2,plazo3)        #Creación y cálculo de la columna ROI30

In [117]:
df2 = df_tether

df_tether['ROI30'] = valorROI(df2,plazo1)        #Creación y cálculo de la columna ROI30
df_tether['ROI90'] = valorROI(df2,plazo2)        #Creación y cálculo de la columna ROI90
df_tether['ROI180'] = valorROI(df2,plazo3)        #Creación y cálculo de la columna ROI180

In [118]:
df2 = df_usd_coin

df_usd_coin['ROI30'] = valorROI(df2,plazo1)        #Creación y cálculo de la columna ROI30
df_usd_coin['ROI90'] = valorROI(df2,plazo2)        #Creación y cálculo de la columna ROI90
df_usd_coin['ROI180'] = valorROI(df2,plazo3)        #Creación y cálculo de la columna ROI180

In [119]:
df2 = df_ethereum

df_ethereum['ROI30'] = valorROI(df2,plazo1)        #Creación y cálculo de la columna ROI30
df_ethereum['ROI90'] = valorROI(df2,plazo2)        #Creación y cálculo de la columna ROI90
df_ethereum['ROI180'] = valorROI(df2,plazo3)        #Creación y cálculo de la columna ROI180

In [120]:
df2 = df_cardano

df_cardano['ROI30'] = valorROI(df2,plazo1)        #Creación y cálculo de la columna ROI30
df_cardano['ROI90'] = valorROI(df2,plazo2)        #Creación y cálculo de la columna ROI90
df_cardano['ROI180'] = valorROI(df2,plazo3)        #Creación y cálculo de la columna ROI180

In [121]:
df2 = df_avalanche

df_avalanche['ROI30'] = valorROI(df2,plazo1)        #Creación y cálculo de la columna ROI30
df_avalanche['ROI90'] = valorROI(df2,plazo2)        #Creación y cálculo de la columna ROI90
df_avalanche['ROI180'] = valorROI(df2,plazo3)        #Creación y cálculo de la columna ROI180

In [122]:
df2 = df_bnb

df_bnb['ROI30'] = valorROI(df2,plazo1)        #Creación y cálculo de la columna ROI30
df_bnb['ROI90'] = valorROI(df2,plazo2)        #Creación y cálculo de la columna ROI90
df_bnb['ROI180'] = valorROI(df2,plazo3)        #Creación y cálculo de la columna ROI180

In [123]:
df2 = df_dogecoin

df_dogecoin['ROI30'] = valorROI(df2,plazo1)        #Creación y cálculo de la columna ROI30
df_dogecoin['ROI90'] = valorROI(df2,plazo2)        #Creación y cálculo de la columna ROI90
df_dogecoin['ROI180'] = valorROI(df2,plazo3)        #Creación y cálculo de la columna ROI180

In [124]:
df2 = df_eos

df_eos['ROI30'] = valorROI(df2,plazo1)        #Creación y cálculo de la columna ROI30
df_eos['ROI90'] = valorROI(df2,plazo2)        #Creación y cálculo de la columna ROI90
df_eos['ROI180'] = valorROI(df2,plazo3)        #Creación y cálculo de la columna ROI180

5.f Consulta de info

In [125]:
df_solana.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     365 non-null    int64  
 1   price         365 non-null    float64
 2   market_cap    365 non-null    float64
 3   total_volume  365 non-null    float64
 4   date          365 non-null    object 
 5   nombre        365 non-null    object 
 6   EMA12         365 non-null    float64
 7   EMA26         365 non-null    float64
 8   EMA200        365 non-null    float64
 9   MACD          365 non-null    float64
 10  signal_line   357 non-null    float64
 11  price_change  364 non-null    float64
 12  gain          365 non-null    float64
 13  loss          365 non-null    float64
 14  ema_gain      365 non-null    float64
 15  ema_loss      365 non-null    float64
 16  rs            364 non-null    float64
 17  rsi           364 non-null    float64
 18  ROI30         365 non-null    

In [126]:
df_bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     365 non-null    int64  
 1   price         365 non-null    float64
 2   market_cap    365 non-null    float64
 3   total_volume  365 non-null    float64
 4   date          365 non-null    object 
 5   nombre        365 non-null    object 
 6   EMA12         365 non-null    float64
 7   EMA26         365 non-null    float64
 8   EMA200        365 non-null    float64
 9   MACD          365 non-null    float64
 10  signal_line   357 non-null    float64
 11  price_change  364 non-null    float64
 12  gain          365 non-null    float64
 13  loss          365 non-null    float64
 14  ema_gain      365 non-null    float64
 15  ema_loss      365 non-null    float64
 16  rs            364 non-null    float64
 17  rsi           364 non-null    float64
 18  ROI30         365 non-null    

In [127]:
df_tether.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     365 non-null    int64  
 1   price         365 non-null    float64
 2   market_cap    365 non-null    float64
 3   total_volume  365 non-null    float64
 4   date          365 non-null    object 
 5   nombre        365 non-null    object 
 6   EMA12         365 non-null    float64
 7   EMA26         365 non-null    float64
 8   EMA200        365 non-null    float64
 9   MACD          365 non-null    float64
 10  signal_line   357 non-null    float64
 11  price_change  364 non-null    float64
 12  gain          365 non-null    float64
 13  loss          365 non-null    float64
 14  ema_gain      365 non-null    float64
 15  ema_loss      365 non-null    float64
 16  rs            364 non-null    float64
 17  rsi           364 non-null    float64
 18  ROI30         365 non-null    

In [128]:
df_usd_coin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     365 non-null    int64  
 1   price         365 non-null    float64
 2   market_cap    365 non-null    float64
 3   total_volume  365 non-null    float64
 4   date          365 non-null    object 
 5   nombre        365 non-null    object 
 6   EMA12         365 non-null    float64
 7   EMA26         365 non-null    float64
 8   EMA200        365 non-null    float64
 9   MACD          365 non-null    float64
 10  signal_line   357 non-null    float64
 11  price_change  364 non-null    float64
 12  gain          365 non-null    float64
 13  loss          365 non-null    float64
 14  ema_gain      365 non-null    float64
 15  ema_loss      365 non-null    float64
 16  rs            364 non-null    float64
 17  rsi           364 non-null    float64
 18  ROI30         365 non-null    

In [129]:
df_ethereum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     365 non-null    int64  
 1   price         365 non-null    float64
 2   market_cap    365 non-null    float64
 3   total_volume  365 non-null    float64
 4   date          365 non-null    object 
 5   nombre        365 non-null    object 
 6   EMA12         365 non-null    float64
 7   EMA26         365 non-null    float64
 8   EMA200        365 non-null    float64
 9   MACD          365 non-null    float64
 10  signal_line   357 non-null    float64
 11  price_change  364 non-null    float64
 12  gain          365 non-null    float64
 13  loss          365 non-null    float64
 14  ema_gain      365 non-null    float64
 15  ema_loss      365 non-null    float64
 16  rs            364 non-null    float64
 17  rsi           364 non-null    float64
 18  ROI30         365 non-null    

In [130]:
df_cardano.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     365 non-null    int64  
 1   price         365 non-null    float64
 2   market_cap    365 non-null    float64
 3   total_volume  365 non-null    float64
 4   date          365 non-null    object 
 5   nombre        365 non-null    object 
 6   EMA12         365 non-null    float64
 7   EMA26         365 non-null    float64
 8   EMA200        365 non-null    float64
 9   MACD          365 non-null    float64
 10  signal_line   357 non-null    float64
 11  price_change  364 non-null    float64
 12  gain          365 non-null    float64
 13  loss          365 non-null    float64
 14  ema_gain      365 non-null    float64
 15  ema_loss      365 non-null    float64
 16  rs            364 non-null    float64
 17  rsi           364 non-null    float64
 18  ROI30         365 non-null    

In [131]:
df_avalanche.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     365 non-null    int64  
 1   price         365 non-null    float64
 2   market_cap    365 non-null    float64
 3   total_volume  365 non-null    float64
 4   date          365 non-null    object 
 5   nombre        365 non-null    object 
 6   EMA12         365 non-null    float64
 7   EMA26         365 non-null    float64
 8   EMA200        365 non-null    float64
 9   MACD          365 non-null    float64
 10  signal_line   357 non-null    float64
 11  price_change  364 non-null    float64
 12  gain          365 non-null    float64
 13  loss          365 non-null    float64
 14  ema_gain      365 non-null    float64
 15  ema_loss      365 non-null    float64
 16  rs            364 non-null    float64
 17  rsi           364 non-null    float64
 18  ROI30         365 non-null    

In [132]:
df_bnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     365 non-null    int64  
 1   price         365 non-null    float64
 2   market_cap    365 non-null    float64
 3   total_volume  365 non-null    float64
 4   date          365 non-null    object 
 5   nombre        365 non-null    object 
 6   EMA12         365 non-null    float64
 7   EMA26         365 non-null    float64
 8   EMA200        365 non-null    float64
 9   MACD          365 non-null    float64
 10  signal_line   357 non-null    float64
 11  price_change  364 non-null    float64
 12  gain          365 non-null    float64
 13  loss          365 non-null    float64
 14  ema_gain      365 non-null    float64
 15  ema_loss      365 non-null    float64
 16  rs            364 non-null    float64
 17  rsi           364 non-null    float64
 18  ROI30         365 non-null    

In [133]:
df_dogecoin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     365 non-null    int64  
 1   price         365 non-null    float64
 2   market_cap    365 non-null    float64
 3   total_volume  365 non-null    float64
 4   date          365 non-null    object 
 5   nombre        365 non-null    object 
 6   EMA12         365 non-null    float64
 7   EMA26         365 non-null    float64
 8   EMA200        365 non-null    float64
 9   MACD          365 non-null    float64
 10  signal_line   357 non-null    float64
 11  price_change  364 non-null    float64
 12  gain          365 non-null    float64
 13  loss          365 non-null    float64
 14  ema_gain      365 non-null    float64
 15  ema_loss      365 non-null    float64
 16  rs            364 non-null    float64
 17  rsi           364 non-null    float64
 18  ROI30         365 non-null    

In [134]:
df_eos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     365 non-null    int64  
 1   price         365 non-null    float64
 2   market_cap    365 non-null    float64
 3   total_volume  365 non-null    float64
 4   date          365 non-null    object 
 5   nombre        365 non-null    object 
 6   EMA12         365 non-null    float64
 7   EMA26         365 non-null    float64
 8   EMA200        365 non-null    float64
 9   MACD          365 non-null    float64
 10  signal_line   357 non-null    float64
 11  price_change  364 non-null    float64
 12  gain          365 non-null    float64
 13  loss          365 non-null    float64
 14  ema_gain      365 non-null    float64
 15  ema_loss      365 non-null    float64
 16  rs            364 non-null    float64
 17  rsi           364 non-null    float64
 18  ROI30         365 non-null    

In [135]:
# Lista de nombres de columnas a eliminar
columnas_a_eliminar = ['timestamp','price_change', 'gain', 'loss', 'ema_gain', 'ema_loss', 'rs']

In [136]:
# Eliminar las columnas especificadas
df_solana = df_solana.drop(columns=columnas_a_eliminar)

In [137]:
df_solana.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         365 non-null    float64
 1   market_cap    365 non-null    float64
 2   total_volume  365 non-null    float64
 3   date          365 non-null    object 
 4   nombre        365 non-null    object 
 5   EMA12         365 non-null    float64
 6   EMA26         365 non-null    float64
 7   EMA200        365 non-null    float64
 8   MACD          365 non-null    float64
 9   signal_line   357 non-null    float64
 10  rsi           364 non-null    float64
 11  ROI30         365 non-null    float64
 12  ROI90         365 non-null    float64
 13  ROI180        365 non-null    float64
dtypes: float64(12), object(2)
memory usage: 40.1+ KB


In [138]:
# Eliminar las columnas especificadas
df_bitcoin = df_bitcoin.drop(columns=columnas_a_eliminar)

In [139]:
df_bitcoin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         365 non-null    float64
 1   market_cap    365 non-null    float64
 2   total_volume  365 non-null    float64
 3   date          365 non-null    object 
 4   nombre        365 non-null    object 
 5   EMA12         365 non-null    float64
 6   EMA26         365 non-null    float64
 7   EMA200        365 non-null    float64
 8   MACD          365 non-null    float64
 9   signal_line   357 non-null    float64
 10  rsi           364 non-null    float64
 11  ROI30         365 non-null    float64
 12  ROI90         365 non-null    float64
 13  ROI180        365 non-null    float64
dtypes: float64(12), object(2)
memory usage: 40.1+ KB


In [140]:
# Eliminar las columnas especificadas
df_tether = df_tether.drop(columns=columnas_a_eliminar)

In [141]:
df_tether.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         365 non-null    float64
 1   market_cap    365 non-null    float64
 2   total_volume  365 non-null    float64
 3   date          365 non-null    object 
 4   nombre        365 non-null    object 
 5   EMA12         365 non-null    float64
 6   EMA26         365 non-null    float64
 7   EMA200        365 non-null    float64
 8   MACD          365 non-null    float64
 9   signal_line   357 non-null    float64
 10  rsi           364 non-null    float64
 11  ROI30         365 non-null    float64
 12  ROI90         365 non-null    float64
 13  ROI180        365 non-null    float64
dtypes: float64(12), object(2)
memory usage: 40.1+ KB


In [142]:
# Eliminar las columnas especificadas
df_usd_coin = df_usd_coin.drop(columns=columnas_a_eliminar)

In [143]:
df_usd_coin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         365 non-null    float64
 1   market_cap    365 non-null    float64
 2   total_volume  365 non-null    float64
 3   date          365 non-null    object 
 4   nombre        365 non-null    object 
 5   EMA12         365 non-null    float64
 6   EMA26         365 non-null    float64
 7   EMA200        365 non-null    float64
 8   MACD          365 non-null    float64
 9   signal_line   357 non-null    float64
 10  rsi           364 non-null    float64
 11  ROI30         365 non-null    float64
 12  ROI90         365 non-null    float64
 13  ROI180        365 non-null    float64
dtypes: float64(12), object(2)
memory usage: 40.1+ KB


In [144]:
# Eliminar las columnas especificadas
df_ethereum = df_ethereum.drop(columns=columnas_a_eliminar)

In [145]:
df_ethereum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         365 non-null    float64
 1   market_cap    365 non-null    float64
 2   total_volume  365 non-null    float64
 3   date          365 non-null    object 
 4   nombre        365 non-null    object 
 5   EMA12         365 non-null    float64
 6   EMA26         365 non-null    float64
 7   EMA200        365 non-null    float64
 8   MACD          365 non-null    float64
 9   signal_line   357 non-null    float64
 10  rsi           364 non-null    float64
 11  ROI30         365 non-null    float64
 12  ROI90         365 non-null    float64
 13  ROI180        365 non-null    float64
dtypes: float64(12), object(2)
memory usage: 40.1+ KB


In [146]:
# Eliminar las columnas especificadas
df_cardano = df_cardano.drop(columns=columnas_a_eliminar)

In [147]:
df_cardano.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         365 non-null    float64
 1   market_cap    365 non-null    float64
 2   total_volume  365 non-null    float64
 3   date          365 non-null    object 
 4   nombre        365 non-null    object 
 5   EMA12         365 non-null    float64
 6   EMA26         365 non-null    float64
 7   EMA200        365 non-null    float64
 8   MACD          365 non-null    float64
 9   signal_line   357 non-null    float64
 10  rsi           364 non-null    float64
 11  ROI30         365 non-null    float64
 12  ROI90         365 non-null    float64
 13  ROI180        365 non-null    float64
dtypes: float64(12), object(2)
memory usage: 40.1+ KB


In [148]:
# Eliminar las columnas especificadas
df_avalanche = df_avalanche.drop(columns=columnas_a_eliminar)

In [149]:
df_avalanche.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         365 non-null    float64
 1   market_cap    365 non-null    float64
 2   total_volume  365 non-null    float64
 3   date          365 non-null    object 
 4   nombre        365 non-null    object 
 5   EMA12         365 non-null    float64
 6   EMA26         365 non-null    float64
 7   EMA200        365 non-null    float64
 8   MACD          365 non-null    float64
 9   signal_line   357 non-null    float64
 10  rsi           364 non-null    float64
 11  ROI30         365 non-null    float64
 12  ROI90         365 non-null    float64
 13  ROI180        365 non-null    float64
dtypes: float64(12), object(2)
memory usage: 40.1+ KB


In [150]:
# Eliminar las columnas especificadas
df_bnb = df_bnb.drop(columns=columnas_a_eliminar)

In [151]:
df_bnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         365 non-null    float64
 1   market_cap    365 non-null    float64
 2   total_volume  365 non-null    float64
 3   date          365 non-null    object 
 4   nombre        365 non-null    object 
 5   EMA12         365 non-null    float64
 6   EMA26         365 non-null    float64
 7   EMA200        365 non-null    float64
 8   MACD          365 non-null    float64
 9   signal_line   357 non-null    float64
 10  rsi           364 non-null    float64
 11  ROI30         365 non-null    float64
 12  ROI90         365 non-null    float64
 13  ROI180        365 non-null    float64
dtypes: float64(12), object(2)
memory usage: 40.1+ KB


In [152]:
# Eliminar las columnas especificadas
df_dogecoin = df_dogecoin.drop(columns=columnas_a_eliminar)

In [153]:
df_dogecoin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         365 non-null    float64
 1   market_cap    365 non-null    float64
 2   total_volume  365 non-null    float64
 3   date          365 non-null    object 
 4   nombre        365 non-null    object 
 5   EMA12         365 non-null    float64
 6   EMA26         365 non-null    float64
 7   EMA200        365 non-null    float64
 8   MACD          365 non-null    float64
 9   signal_line   357 non-null    float64
 10  rsi           364 non-null    float64
 11  ROI30         365 non-null    float64
 12  ROI90         365 non-null    float64
 13  ROI180        365 non-null    float64
dtypes: float64(12), object(2)
memory usage: 40.1+ KB


In [154]:
# Eliminar las columnas especificadas
df_eos = df_eos.drop(columns=columnas_a_eliminar)

In [155]:
df_eos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         365 non-null    float64
 1   market_cap    365 non-null    float64
 2   total_volume  365 non-null    float64
 3   date          365 non-null    object 
 4   nombre        365 non-null    object 
 5   EMA12         365 non-null    float64
 6   EMA26         365 non-null    float64
 7   EMA200        365 non-null    float64
 8   MACD          365 non-null    float64
 9   signal_line   357 non-null    float64
 10  rsi           364 non-null    float64
 11  ROI30         365 non-null    float64
 12  ROI90         365 non-null    float64
 13  ROI180        365 non-null    float64
dtypes: float64(12), object(2)
memory usage: 40.1+ KB


5.f Consulta de nulos

In [156]:
df_solana.isna().sum()

price           0
market_cap      0
total_volume    0
date            0
nombre          0
EMA12           0
EMA26           0
EMA200          0
MACD            0
signal_line     8
rsi             1
ROI30           0
ROI90           0
ROI180          0
dtype: int64

In [157]:
filas_nulas = df_solana[df_solana.isna().any(axis=1)]

In [158]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,43.902581,1.532266e+10,1.018587e+09,15-08-2022,Solana,43.902581,43.902581,43.902581,0.000000,NaN,NaN,-8.05682,21.567564,5.268193
1,43.252084,1.506556e+10,1.075538e+09,16-08-2022,Solana,43.802505,43.854396,43.896109,-0.051892,NaN,0.000000,-8.05682,21.567564,5.268193
2,40.556286,1.416116e+10,1.249878e+09,17-08-2022,Solana,43.303086,43.610092,43.862877,-0.307005,NaN,0.000000,-8.05682,21.567564,5.268193
3,39.618895,1.386326e+10,8.249718e+08,18-08-2022,Solana,42.736288,43.314448,43.820648,-0.578160,NaN,0.000000,-8.05682,21.567564,5.268193
4,35.989795,1.253047e+10,1.786764e+09,19-08-2022,Solana,41.698366,42.771881,43.742729,-1.073515,NaN,0.000000,-8.05682,21.567564,5.268193
5,35.231830,1.225669e+10,1.200537e+09,20-08-2022,Solana,40.703514,42.213358,43.658043,-1.509844,NaN,0.000000,-8.05682,21.567564,5.268193
6,36.532724,1.281531e+10,9.912247e+08,21-08-2022,Solana,40.061854,41.792571,43.587145,-1.730717,NaN,18.238191,-8.05682,21.567564,5.268193
7,35.489795,1.235258e+10,1.122988e+09,22-08-2022,Solana,39.358460,41.325698,43.506574,-1.967238,NaN,15.605395,-8.05682,21.567564,5.268193


Los registros son de hace un año, por los que los descartaremos ya que no influyen en nuestro análisis

In [159]:
df_bitcoin.isna().sum()

price           0
market_cap      0
total_volume    0
date            0
nombre          0
EMA12           0
EMA26           0
EMA200          0
MACD            0
signal_line     8
rsi             1
ROI30           0
ROI90           0
ROI180          0
dtype: int64

In [160]:
filas_nulas = df_bitcoin[df_bitcoin.isna().any(axis=1)]

In [161]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,24179.014652,4.624372e+11,3.340481e+10,15-08-2022,Bitcoin,24179.014652,24179.014652,24179.014652,0.000000,NaN,NaN,-2.961864,8.799537,20.950872
1,23913.333726,4.567910e+11,2.497130e+10,16-08-2022,Bitcoin,24138.140664,24159.334584,24176.371061,-21.193920,NaN,0.000000,-2.961864,8.799537,20.950872
2,23359.418837,4.470158e+11,2.956740e+10,17-08-2022,Bitcoin,24018.337306,24100.081565,24168.242183,-81.744260,NaN,0.000000,-2.961864,8.799537,20.950872
3,23248.384228,4.448460e+11,2.022821e+10,18-08-2022,Bitcoin,23899.882986,24036.992874,24159.089368,-137.109888,NaN,0.000000,-2.961864,8.799537,20.950872
4,20945.833966,4.008104e+11,3.514241e+10,19-08-2022,Bitcoin,23445.413906,23808.018140,24127.116677,-362.604234,NaN,0.000000,-2.961864,8.799537,20.950872
5,21175.232733,4.049249e+11,2.380968e+10,20-08-2022,Bitcoin,23096.155264,23612.996999,24097.744698,-516.841735,NaN,8.138147,-2.961864,8.799537,20.950872
6,21615.761693,4.135789e+11,2.047624e+10,21-08-2022,Bitcoin,22868.402407,23465.053643,24073.048349,-596.651236,NaN,22.172431,-2.961864,8.799537,20.950872
7,21387.747115,4.084091e+11,2.885691e+10,22-08-2022,Bitcoin,22640.609285,23311.179085,24046.328934,-670.569800,NaN,20.318543,-2.961864,8.799537,20.950872


Los registros son de hace un año, por los que los descartaremos ya que no influyen en nuestro análisis

In [162]:
df_tether.isna().sum()

price           0
market_cap      0
total_volume    0
date            0
nombre          0
EMA12           0
EMA26           0
EMA200          0
MACD            0
signal_line     8
rsi             1
ROI30           0
ROI90           0
ROI180          0
dtype: int64

In [163]:
filas_nulas = df_tether[df_tether.isna().any(axis=1)]

In [164]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,1.001842,6.765299e+10,5.762541e+10,15-08-2022,Tether,1.001842,1.001842,1.001842,0.000000,NaN,NaN,-0.193917,-0.1441,-0.112112
1,1.000989,6.762839e+10,4.820835e+10,16-08-2022,Tether,1.001711,1.001779,1.001833,-0.000068,NaN,0.000000,-0.193917,-0.1441,-0.112112
2,1.000875,6.766131e+10,5.435838e+10,17-08-2022,Tether,1.001582,1.001712,1.001824,-0.000130,NaN,0.000000,-0.193917,-0.1441,-0.112112
3,1.001241,6.759940e+10,4.209014e+10,18-08-2022,Tether,1.001530,1.001677,1.001818,-0.000147,NaN,33.073016,-0.193917,-0.1441,-0.112112
4,1.003335,6.761379e+10,6.330288e+10,19-08-2022,Tether,1.001807,1.001800,1.001833,0.000008,NaN,79.011925,-0.193917,-0.1441,-0.112112
5,1.000816,6.763657e+10,4.623376e+10,20-08-2022,Tether,1.001655,1.001727,1.001823,-0.000072,NaN,40.470716,-0.193917,-0.1441,-0.112112
6,1.002970,6.770997e+10,4.225511e+10,21-08-2022,Tether,1.001857,1.001819,1.001834,0.000038,NaN,59.808647,-0.193917,-0.1441,-0.112112
7,1.000702,6.745998e+10,5.054864e+10,22-08-2022,Tether,1.001679,1.001736,1.001823,-0.000057,NaN,42.880587,-0.193917,-0.1441,-0.112112


Los registros son de hace un año, por los que los descartaremos ya que no influyen en nuestro análisis

In [165]:
df_usd_coin.isna().sum()

price           0
market_cap      0
total_volume    0
date            0
nombre          0
EMA12           0
EMA26           0
EMA200          0
MACD            0
signal_line     8
rsi             1
ROI30           0
ROI90           0
ROI180          0
dtype: int64

In [166]:
filas_nulas = df_usd_coin[df_usd_coin.isna().any(axis=1)]

In [167]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,1.001911,5.358637e+10,7.323677e+09,15-08-2022,USD Coin,1.001911,1.001911,1.001911,0.000000,NaN,NaN,-0.043382,-0.017221,-0.028772
1,1.001137,5.356327e+10,6.057253e+09,16-08-2022,USD Coin,1.001792,1.001854,1.001904,-0.000062,NaN,0.000000,-0.043382,-0.017221,-0.028772
2,1.000239,5.319273e+10,6.330005e+09,17-08-2022,USD Coin,1.001553,1.001734,1.001887,-0.000181,NaN,0.000000,-0.043382,-0.017221,-0.028772
3,1.000279,5.288017e+10,5.127734e+09,18-08-2022,USD Coin,1.001357,1.001627,1.001871,-0.000269,NaN,2.857538,-0.043382,-0.017221,-0.028772
4,1.002019,5.252307e+10,7.381892e+09,19-08-2022,USD Coin,1.001459,1.001656,1.001873,-0.000197,NaN,60.090830,-0.043382,-0.017221,-0.028772
5,0.999737,5.243786e+10,5.008903e+09,20-08-2022,USD Coin,1.001194,1.001514,1.001851,-0.000319,NaN,31.771919,-0.043382,-0.017221,-0.028772
6,1.002564,5.265940e+10,5.209885e+09,21-08-2022,USD Coin,1.001405,1.001591,1.001858,-0.000186,NaN,59.237590,-0.043382,-0.017221,-0.028772
7,1.001651,5.234006e+10,5.920946e+09,22-08-2022,USD Coin,1.001443,1.001596,1.001856,-0.000153,NaN,51.505424,-0.043382,-0.017221,-0.028772


Los registros son de hace un año, por los que los descartaremos ya que no influyen en nuestro análisis

In [168]:
df_ethereum.isna().sum()

price           0
market_cap      0
total_volume    0
date            0
nombre          0
EMA12           0
EMA26           0
EMA200          0
MACD            0
signal_line     8
rsi             1
ROI30           0
ROI90           0
ROI180          0
dtype: int64

In [169]:
filas_nulas = df_ethereum[df_ethereum.isna().any(axis=1)]

In [170]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,1908.277642,2.294404e+11,1.831087e+10,15-08-2022,Ethereum,1908.277642,1908.277642,1908.277642,0.000000,NaN,NaN,-4.549885,1.088489,10.069819
1,1880.600101,2.255816e+11,1.424023e+10,16-08-2022,Ethereum,1904.019559,1906.227454,1908.002243,-2.207895,NaN,0.000000,-4.549885,1.088489,10.069819
2,1834.826453,2.206659e+11,1.842755e+10,17-08-2022,Ethereum,1893.374465,1900.938491,1907.274126,-7.564025,NaN,0.000000,-4.549885,1.088489,10.069819
3,1850.111291,2.226524e+11,1.336378e+10,18-08-2022,Ethereum,1886.718592,1897.173513,1906.705342,-10.454920,NaN,20.179538,-4.549885,1.088489,10.069819
4,1618.532504,1.948177e+11,2.268736e+10,19-08-2022,Ethereum,1845.459194,1876.533438,1903.837950,-31.074244,NaN,4.456864,-4.549885,1.088489,10.069819
5,1579.415723,1.896211e+11,1.640451e+10,20-08-2022,Ethereum,1804.529430,1854.524719,1900.609868,-49.995289,NaN,3.869294,-4.549885,1.088489,10.069819
6,1624.149983,1.953245e+11,1.417470e+10,21-08-2022,Ethereum,1776.778745,1837.459923,1897.859024,-60.681178,NaN,18.114309,-4.549885,1.088489,10.069819
7,1622.307035,1.937082e+11,1.664864e+10,22-08-2022,Ethereum,1753.013867,1821.522672,1895.117213,-68.508805,NaN,17.987604,-4.549885,1.088489,10.069819


Los registros son de hace un año, por los que los descartaremos ya que no influyen en nuestro análisis

In [171]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,1908.277642,2.294404e+11,1.831087e+10,15-08-2022,Ethereum,1908.277642,1908.277642,1908.277642,0.000000,NaN,NaN,-4.549885,1.088489,10.069819
1,1880.600101,2.255816e+11,1.424023e+10,16-08-2022,Ethereum,1904.019559,1906.227454,1908.002243,-2.207895,NaN,0.000000,-4.549885,1.088489,10.069819
2,1834.826453,2.206659e+11,1.842755e+10,17-08-2022,Ethereum,1893.374465,1900.938491,1907.274126,-7.564025,NaN,0.000000,-4.549885,1.088489,10.069819
3,1850.111291,2.226524e+11,1.336378e+10,18-08-2022,Ethereum,1886.718592,1897.173513,1906.705342,-10.454920,NaN,20.179538,-4.549885,1.088489,10.069819
4,1618.532504,1.948177e+11,2.268736e+10,19-08-2022,Ethereum,1845.459194,1876.533438,1903.837950,-31.074244,NaN,4.456864,-4.549885,1.088489,10.069819
5,1579.415723,1.896211e+11,1.640451e+10,20-08-2022,Ethereum,1804.529430,1854.524719,1900.609868,-49.995289,NaN,3.869294,-4.549885,1.088489,10.069819
6,1624.149983,1.953245e+11,1.417470e+10,21-08-2022,Ethereum,1776.778745,1837.459923,1897.859024,-60.681178,NaN,18.114309,-4.549885,1.088489,10.069819
7,1622.307035,1.937082e+11,1.664864e+10,22-08-2022,Ethereum,1753.013867,1821.522672,1895.117213,-68.508805,NaN,17.987604,-4.549885,1.088489,10.069819


In [172]:
df_cardano.isna().sum()

price           0
market_cap      0
total_volume    0
date            0
nombre          0
EMA12           0
EMA26           0
EMA200          0
MACD            0
signal_line     8
rsi             1
ROI30           0
ROI90           0
ROI180          0
dtype: int64

In [173]:
filas_nulas = df_cardano[df_cardano.isna().any(axis=1)]

In [174]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,0.552459,1.869741e+10,8.236799e+08,15-08-2022,Cardano,0.552459,0.552459,0.552459,0.000000,NaN,NaN,-10.887525,-21.091502,-29.687205
1,0.559785,1.892784e+10,6.807133e+08,16-08-2022,Cardano,0.553586,0.553001,0.552532,0.000584,NaN,100.000000,-10.887525,-21.091502,-29.687205
2,0.539302,1.821255e+10,9.965319e+08,17-08-2022,Cardano,0.551388,0.551987,0.552400,-0.000598,NaN,23.663013,-10.887525,-21.091502,-29.687205
3,0.514487,1.741240e+10,5.741756e+08,18-08-2022,Cardano,0.545711,0.549209,0.552023,-0.003498,NaN,11.447644,-10.887525,-21.091502,-29.687205
4,0.452547,1.526462e+10,1.131870e+09,19-08-2022,Cardano,0.531378,0.542049,0.551033,-0.010671,NaN,4.603419,-10.887525,-21.091502,-29.687205
5,0.452165,1.528737e+10,7.126051e+08,20-08-2022,Cardano,0.519192,0.535391,0.550049,-0.016199,NaN,4.583918,-10.887525,-21.091502,-29.687205
6,0.464937,1.573708e+10,5.729422e+08,21-08-2022,Cardano,0.510845,0.530172,0.549202,-0.019327,NaN,17.987859,-10.887525,-21.091502,-29.687205
7,0.461040,1.555897e+10,6.523654e+08,22-08-2022,Cardano,0.503182,0.525051,0.548325,-0.021869,NaN,17.140018,-10.887525,-21.091502,-29.687205


Los registros son de hace un año, por los que los descartaremos ya que no influyen en nuestro análisis

In [175]:
df_avalanche.isna().sum()

price           0
market_cap      0
total_volume    0
date            0
nombre          0
EMA12           0
EMA26           0
EMA200          0
MACD            0
signal_line     8
rsi             1
ROI30           0
ROI90           0
ROI180          0
dtype: int64

In [176]:
filas_nulas = df_avalanche[df_avalanche.isna().any(axis=1)]

In [177]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,27.878861,7.904216e+09,4.390438e+08,15-08-2022,Avalanche,27.878861,27.878861,27.878861,0.000000,NaN,NaN,-16.692952,-17.098976,-37.873332
1,27.251164,7.763571e+09,3.659226e+08,16-08-2022,Avalanche,27.782293,27.832365,27.872616,-0.050073,NaN,0.000000,-16.692952,-17.098976,-37.873332
2,25.530010,7.283666e+09,4.260619e+08,17-08-2022,Avalanche,27.435788,27.661820,27.849306,-0.226033,NaN,0.000000,-16.692952,-17.098976,-37.873332
3,24.299012,6.932924e+09,3.173880e+08,18-08-2022,Avalanche,26.953207,27.412724,27.813980,-0.459517,NaN,0.000000,-16.692952,-17.098976,-37.873332
4,22.535468,6.421005e+09,6.394015e+08,19-08-2022,Avalanche,26.273555,27.051445,27.761457,-0.777891,NaN,0.000000,-16.692952,-17.098976,-37.873332
5,22.276532,6.331474e+09,4.377654e+08,20-08-2022,Avalanche,25.658628,26.697748,27.706881,-1.039120,NaN,0.000000,-16.692952,-17.098976,-37.873332
6,23.063913,6.580656e+09,3.309412e+08,21-08-2022,Avalanche,25.259441,26.428575,27.660682,-1.169134,NaN,17.831496,-16.692952,-17.098976,-37.873332
7,22.578587,6.414941e+09,4.797824e+08,22-08-2022,Avalanche,24.847002,26.143391,27.610114,-1.296389,NaN,15.824632,-16.692952,-17.098976,-37.873332


Los registros son de hace un año, por los que los descartaremos ya que no influyen en nuestro análisis

In [178]:
df_bnb.isna().sum()

price           0
market_cap      0
total_volume    0
date            0
nombre          0
EMA12           0
EMA26           0
EMA200          0
MACD            0
signal_line     8
rsi             1
ROI30           0
ROI90           0
ROI180          0
dtype: int64

In [179]:
filas_nulas = df_bnb[df_bnb.isna().any(axis=1)]

In [180]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,319.974835,5.225906e+10,1.029621e+09,15-08-2022,BNB,319.974835,319.974835,319.974835,0.000000,NaN,NaN,-4.275281,-22.82841,-24.237907
1,316.698400,5.164471e+10,1.009809e+09,16-08-2022,BNB,319.470769,319.732137,319.942234,-0.261368,NaN,0.000000,-4.275281,-22.82841,-24.237907
2,306.639287,5.010061e+10,1.199734e+09,17-08-2022,BNB,317.496694,318.762296,319.809866,-1.265601,NaN,0.000000,-4.275281,-22.82841,-24.237907
3,298.600700,4.882570e+10,1.091856e+09,18-08-2022,BNB,314.589618,317.268844,319.598830,-2.679226,NaN,0.000000,-4.275281,-22.82841,-24.237907
4,279.977814,4.564295e+10,1.657993e+09,19-08-2022,BNB,309.264725,314.506546,319.204591,-5.241820,NaN,0.000000,-4.275281,-22.82841,-24.237907
5,283.656155,4.625237e+10,1.246838e+09,20-08-2022,BNB,305.324945,312.221332,318.850875,-6.896386,NaN,10.738862,-4.275281,-22.82841,-24.237907
6,303.015148,4.948623e+10,1.195298e+09,21-08-2022,BNB,304.969592,311.539392,318.693306,-6.569800,NaN,45.972219,-4.275281,-22.82841,-24.237907
7,301.156136,4.915049e+10,1.570720e+09,22-08-2022,BNB,304.382906,310.770262,318.518807,-6.387356,NaN,44.045831,-4.275281,-22.82841,-24.237907


Los registros son de hace un año, por los que los descartaremos ya que no influyen en nuestro análisis

In [181]:
df_dogecoin.isna().sum()

price           0
market_cap      0
total_volume    0
date            0
nombre          0
EMA12           0
EMA26           0
EMA200          0
MACD            0
signal_line     8
rsi             1
ROI30           0
ROI90           0
ROI180          0
dtype: int64

In [182]:
filas_nulas = df_dogecoin[df_dogecoin.isna().any(axis=1)]

In [183]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,0.076939,1.022052e+10,8.810870e+08,15-08-2022,Dogecoin,0.076939,0.076939,0.076939,0.000000,NaN,NaN,3.79153,2.76054,-16.205266
1,0.087191,1.161382e+10,1.907993e+09,16-08-2022,Dogecoin,0.078516,0.077698,0.077041,0.000818,NaN,100.000000,3.79153,2.76054,-16.205266
2,0.080599,1.070143e+10,9.878966e+08,17-08-2022,Dogecoin,0.078837,0.077913,0.077076,0.000923,NaN,57.408936,3.79153,2.76054,-16.205266
3,0.075108,9.988655e+09,5.991804e+08,18-08-2022,Dogecoin,0.078263,0.077705,0.077057,0.000558,NaN,40.733602,3.79153,2.76054,-16.205266
4,0.068219,9.037215e+09,1.015524e+09,19-08-2022,Dogecoin,0.076718,0.077003,0.076969,-0.000285,NaN,28.675834,3.79153,2.76054,-16.205266
5,0.069657,9.234800e+09,5.975917e+08,20-08-2022,Dogecoin,0.075631,0.076459,0.076896,-0.000827,NaN,33.420578,3.79153,2.76054,-16.205266
6,0.069615,9.255550e+09,4.815426e+08,21-08-2022,Dogecoin,0.074706,0.075952,0.076824,-0.001246,NaN,33.347018,3.79153,2.76054,-16.205266
7,0.068487,9.072607e+09,5.831581e+08,22-08-2022,Dogecoin,0.073749,0.075399,0.076741,-0.001650,NaN,31.184237,3.79153,2.76054,-16.205266


Los registros son de hace un año, por los que los descartaremos ya que no influyen en nuestro análisis

In [184]:
df_eos.isna().sum()

price           0
market_cap      0
total_volume    0
date            0
nombre          0
EMA12           0
EMA26           0
EMA200          0
MACD            0
signal_line     8
rsi             1
ROI30           0
ROI90           0
ROI180          0
dtype: int64

In [185]:
filas_nulas = df_eos[df_eos.isna().any(axis=1)]

In [186]:
filas_nulas.head(10)

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,1.275740,1.282014e+09,2.234985e+08,15-08-2022,EOS,1.275740,1.275740,1.275740,0.000000,NaN,NaN,-7.078337,-19.501848,-34.738178
1,1.377330,1.378720e+09,3.101783e+08,16-08-2022,EOS,1.291370,1.283266,1.276751,0.008104,NaN,100.000000,-7.078337,-19.501848,-34.738178
2,1.483177,1.492323e+09,1.224827e+09,17-08-2022,EOS,1.320878,1.298074,1.278805,0.022805,NaN,100.000000,-7.078337,-19.501848,-34.738178
3,1.480666,1.493785e+09,5.873258e+08,18-08-2022,EOS,1.345461,1.311599,1.280814,0.033862,NaN,98.527918,-7.078337,-19.501848,-34.738178
4,1.276770,1.281015e+09,5.529318e+08,19-08-2022,EOS,1.334893,1.309019,1.280774,0.025874,NaN,41.407888,-7.078337,-19.501848,-34.738178
5,1.412295,1.417665e+09,4.247303e+08,20-08-2022,EOS,1.346801,1.316669,1.282082,0.030132,NaN,59.441092,-7.078337,-19.501848,-34.738178
6,1.577695,1.587098e+09,6.753532e+08,21-08-2022,EOS,1.382323,1.336005,1.285024,0.046319,NaN,71.704574,-7.078337,-19.501848,-34.738178
7,1.842758,1.857778e+09,1.397465e+09,22-08-2022,EOS,1.453159,1.373542,1.290573,0.079618,NaN,81.851463,-7.078337,-19.501848,-34.738178


Los registros son de hace un año, por los que los descartaremos ya que no influyen en nuestro análisis

5.g Consulta de datos estadísticos

In [187]:
df_solana.describe()

,price,market_cap,total_volume,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
count,365.000000,3.650000e+02,3.650000e+02,365.000000,365.000000,365.000000,365.000000,357.000000,364.000000,3.650000e+02,3.650000e+02,3.650000e+02
mean,22.753024,8.544441e+09,6.968344e+08,23.048701,23.437734,28.686043,-0.389034,-0.394917,47.036905,-8.056820e+00,2.156756e+01,5.268193e+00
std,6.855501,2.361999e+09,6.175810e+08,6.936083,7.065322,7.389370,1.484766,1.447973,17.935384,1.778795e-15,3.557590e-15,8.893976e-16
min,9.597795,3.537484e+09,1.127131e+08,10.755528,11.908296,21.139769,-4.813522,-4.714393,0.000000,-8.056820e+00,2.156756e+01,5.268193e+00
25%,18.612770,7.414057e+09,3.378071e+08,19.355948,19.246041,23.078585,-1.027537,-1.031291,35.386569,-8.056820e+00,2.156756e+01,5.268193e+00
50%,21.986706,8.543485e+09,5.310565e+08,22.170213,21.952658,25.040691,-0.271898,-0.324564,45.779239,-8.056820e+00,2.156756e+01,5.268193e+00
75%,26.035008,1.017896e+10,8.462960e+08,24.821678,25.169025,35.928100,0.275319,0.287003,58.648612,-8.056820e+00,2.156756e+01,5.268193e+00
max,43.902581,1.532266e+10,5.868071e+09,43.902581,43.902581,43.902581,3.003358,2.834439,95.521943,-8.056820e+00,2.156756e+01,5.268193e+00


In [188]:
df_bitcoin.describe()

,price,market_cap,total_volume,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
count,365.000000,3.650000e+02,3.650000e+02,365.000000,365.000000,365.000000,365.000000,357.000000,364.000000,365.000000,3.650000e+02,365.000000
mean,23493.457521,4.535583e+11,2.378893e+10,23415.158290,23313.267234,22700.136962,101.891055,107.011565,50.727485,-2.961864,8.799537e+00,20.950872
std,4764.658933,9.391756e+10,1.401801e+10,4595.914973,4392.286432,2180.875987,634.480077,617.274287,17.654957,0.000000,1.778795e-15,0.000000
min,15742.444336,3.019638e+11,3.882362e+09,16558.222880,16818.802484,19450.370292,-1051.211481,-1006.251461,0.000000,-2.961864,8.799537e+00,20.950872
25%,19418.037989,3.720756e+11,1.302386e+10,19424.430595,19673.415275,20793.841522,-349.854395,-312.208399,38.129916,-2.961864,8.799537e+00,20.950872
50%,23137.323554,4.448983e+11,2.112492e+10,22918.689396,22745.384703,22478.485431,-83.900737,-74.607450,48.595208,-2.961864,8.799537e+00,20.950872
75%,28048.469979,5.426756e+11,3.135476e+10,27849.665285,27627.160663,24353.518741,557.645389,549.791411,62.326427,-2.961864,8.799537e+00,20.950872
max,31446.014971,6.107255e+11,9.214496e+10,30497.931961,29965.107334,27089.185550,1514.018554,1467.946642,97.487765,-2.961864,8.799537e+00,20.950872


In [189]:
df_tether.describe()

,price,market_cap,total_volume,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
count,365.000000,3.650000e+02,3.650000e+02,365.000000,365.000000,365.000000,365.000000,357.000000,364.000000,365.000000,365.0000,3.650000e+02
mean,1.000586,7.385721e+10,3.299056e+10,1.000626,1.000668,1.000985,-0.000042,-0.000040,49.141626,-0.193917,-0.1441,-1.121119e-01
std,0.001416,7.336519e+09,1.978701e+10,0.000908,0.000756,0.000348,0.000307,0.000280,8.429410,0.000000,0.0000,4.169051e-17
min,0.993587,6.533852e+10,6.894278e+09,0.999056,0.999453,1.000380,-0.000617,-0.000546,0.000000,-0.193917,-0.1441,-1.121119e-01
25%,0.999968,6.759566e+10,1.900308e+10,1.000144,1.000183,1.000683,-0.000201,-0.000182,44.444519,-0.193917,-0.1441,-1.121119e-01
50%,1.000383,6.951115e+10,2.961672e+10,1.000404,1.000450,1.000925,-0.000059,-0.000063,49.005953,-0.193917,-0.1441,-1.121119e-01
75%,1.000894,8.284742e+10,4.334518e+10,1.000858,1.000855,1.001205,0.000047,0.000042,53.596751,-0.193917,-0.1441,-1.121119e-01
max,1.009594,8.389507e+10,1.844138e+11,1.005040,1.003729,1.001842,0.001599,0.001215,79.011925,-0.193917,-0.1441,-1.121119e-01


In [190]:
df_usd_coin.describe()

,price,market_cap,total_volume,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
count,365.000000,3.650000e+02,3.650000e+02,365.000000,365.000000,365.000000,365.000000,357.000000,364.000000,3.650000e+02,365.000000,3.650000e+02
mean,1.000260,3.867318e+10,3.905401e+09,1.000289,1.000324,1.000739,-0.000035,-0.000034,49.319344,-4.338188e-02,-0.017221,-2.877189e-02
std,0.002228,8.384872e+09,2.164082e+09,0.000797,0.000545,0.000461,0.000376,0.000285,8.179522,1.389684e-17,0.000000,6.948419e-18
min,0.965578,2.600436e+10,8.887580e+08,0.994233,0.997224,1.000153,-0.003172,-0.001341,0.000000,-4.338188e-02,-0.017221,-2.877189e-02
25%,0.999749,2.979204e+10,2.525737e+09,0.999992,1.000008,1.000335,-0.000137,-0.000135,46.290386,-4.338188e-02,-0.017221,-2.877189e-02
50%,1.000210,4.232748e+10,3.502420e+09,1.000267,1.000358,1.000705,-0.000029,-0.000027,49.698308,-4.338188e-02,-0.017221,-2.877189e-02
75%,1.000842,4.423378e+10,4.651800e+09,1.000581,1.000590,1.000945,0.000050,0.000040,53.505331,-4.338188e-02,-0.017221,-2.877189e-02
max,1.006530,5.358637e+10,2.186311e+10,1.002487,1.001911,1.001911,0.001410,0.001137,77.244272,-4.338188e-02,-0.017221,-2.877189e-02


In [191]:
df_ethereum.describe()

,price,market_cap,total_volume,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
count,365.000000,3.650000e+02,3.650000e+02,365.000000,365.000000,365.000000,365.000000,357.000000,364.000000,3.650000e+02,3.650000e+02,3.650000e+02
mean,1613.933163,1.943008e+11,1.016965e+10,1614.859106,1615.756880,1647.616452,-0.897774,-0.635867,50.202519,-4.549885e+00,1.088489e+00,1.006982e+01
std,254.069520,3.037299e+10,6.853829e+09,242.706218,232.291730,124.920467,44.727286,43.252614,15.893816,1.778795e-15,2.223494e-16,1.778795e-15
min,1095.183977,1.317944e+11,2.179799e+09,1200.633719,1220.682172,1454.966514,-101.067035,-92.174417,0.000000,-4.549885e+00,1.088489e+00,1.006982e+01
25%,1335.738971,1.609992e+11,6.199278e+09,1369.466691,1401.425459,1523.964854,-23.392346,-23.742750,39.627437,-4.549885e+00,1.088489e+00,1.006982e+01
50%,1646.143502,1.983393e+11,8.731620e+09,1628.507434,1624.861329,1659.086090,-2.445621,-2.835950,48.438692,-4.549885e+00,1.088489e+00,1.006982e+01
75%,1847.541835,2.220698e+11,1.180948e+10,1847.611339,1851.824312,1743.678759,26.303181,26.613394,60.459925,-4.549885e+00,1.088489e+00,1.006982e+01
max,2118.598789,2.535078e+11,6.521171e+10,2007.428637,1918.301903,1908.277642,100.199276,92.967091,96.228619,-4.549885e+00,1.088489e+00,1.006982e+01


In [192]:
df_cardano.describe()

,price,market_cap,total_volume,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
count,365.000000,3.650000e+02,3.650000e+02,365.000000,365.000000,365.000000,365.000000,357.000000,364.000000,3.650000e+02,3.650000e+02,365.000000
mean,0.360457,1.252988e+10,3.758140e+08,0.364337,0.369145,0.418894,-0.004808,-0.004826,45.553205,-1.088752e+01,-2.109150e+01,-29.687205
std,0.063103,2.068890e+09,2.212437e+08,0.063177,0.063778,0.063296,0.012774,0.012418,17.752350,1.778795e-15,3.557590e-15,0.000000
min,0.244315,8.562206e+09,8.908647e+07,0.255177,0.269220,0.338090,-0.029469,-0.027934,4.583918,-1.088752e+01,-2.109150e+01,-29.687205
25%,0.312275,1.094777e+10,2.085889e+08,0.311463,0.313414,0.377848,-0.013723,-0.013802,34.745302,-1.088752e+01,-2.109150e+01,-29.687205
50%,0.362998,1.267902e+10,3.227167e+08,0.368987,0.368840,0.387080,-0.006340,-0.006338,44.571167,-1.088752e+01,-2.109150e+01,-29.687205
75%,0.399433,1.397146e+10,4.716116e+08,0.393836,0.396323,0.471724,0.002881,0.003464,56.688125,-1.088752e+01,-2.109150e+01,-29.687205
max,0.559785,1.892784e+10,1.399406e+09,0.553586,0.553001,0.552532,0.024198,0.023127,100.000000,-1.088752e+01,-2.109150e+01,-29.687205


In [193]:
df_avalanche.describe()

,price,market_cap,total_volume,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
count,365.000000,3.650000e+02,3.650000e+02,365.000000,365.000000,365.000000,365.000000,357.000000,364.000000,3.650000e+02,3.650000e+02,365.000000
mean,15.908008,5.038461e+09,2.368198e+08,16.139409,16.425082,19.433873,-0.285673,-0.284898,44.871493,-1.669295e+01,-1.709898e+01,-37.873332
std,3.057774,8.470897e+08,1.566633e+08,3.137092,3.233457,3.510518,0.753447,0.737141,16.715599,3.557590e-15,3.557590e-15,0.000000
min,10.862361,3.382540e+09,4.765867e+07,11.341802,11.859246,14.944782,-2.127590,-2.059729,0.000000,-1.669295e+01,-1.709898e+01,-37.873332
25%,13.194819,4.402888e+09,1.325819e+08,13.331965,13.551866,17.540093,-0.723765,-0.706945,32.965230,-1.669295e+01,-1.709898e+01,-37.873332
50%,15.948163,5.003360e+09,1.762701e+08,16.182189,16.707052,17.847085,-0.301546,-0.360381,42.738098,-1.669295e+01,-1.709898e+01,-37.873332
75%,17.807151,5.622852e+09,2.948132e+08,17.936287,18.007879,21.752318,0.096840,0.129142,56.852180,-1.669295e+01,-1.709898e+01,-37.873332
max,27.878861,7.904216e+09,1.151507e+09,27.878861,27.878861,27.878861,1.835417,1.776902,91.797093,-1.669295e+01,-1.709898e+01,-37.873332


In [194]:
df_bnb.describe()

,price,market_cap,total_volume,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
count,365.000000,3.650000e+02,3.650000e+02,365.000000,365.000000,365.000000,365.000000,357.000000,364.000000,3.650000e+02,3.650000e+02,3.650000e+02
mean,286.129637,4.531273e+10,7.075647e+08,287.314670,288.798664,298.459084,-1.483995,-1.485454,47.968440,-4.275281e+00,-2.282841e+01,-2.423791e+01
std,30.910085,5.058897e+09,4.840469e+08,27.870319,24.826445,9.578175,7.032946,6.766048,16.517990,8.893976e-16,7.115181e-15,3.557590e-15
min,227.858484,3.593862e+10,1.776448e+07,238.262510,242.039243,274.746210,-20.250498,-19.017496,0.000000,-4.275281e+00,-2.282841e+01,-2.423791e+01
25%,260.405387,4.108703e+10,4.101204e+08,270.656913,275.120753,292.368636,-5.508245,-5.377639,38.038647,-4.275281e+00,-2.282841e+01,-2.423791e+01
50%,288.023270,4.561779e+10,5.939201e+08,289.820059,291.209231,298.989194,-1.498146,-1.639309,45.540631,-4.275281e+00,-2.282841e+01,-2.423791e+01
75%,311.039205,4.915049e+10,8.953504e+08,311.716091,309.932715,305.445139,2.047908,2.187964,59.015132,-4.275281e+00,-2.282841e+01,-2.423791e+01
max,352.946158,5.762904e+10,3.463407e+09,330.281073,326.995184,319.974835,17.319844,14.118605,92.043532,-4.275281e+00,-2.282841e+01,-2.423791e+01


In [195]:
df_dogecoin.describe()

,price,market_cap,total_volume,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
count,365.000000,3.650000e+02,3.650000e+02,365.000000,365.000000,365.000000,365.000000,357.000000,364.000000,365.00000,3.650000e+02,3.650000e+02
mean,0.076591,1.054648e+10,7.790323e+08,0.076617,0.076678,0.077269,-0.000062,-0.000077,47.851977,3.79153,2.760540e+00,-1.620527e+01
std,0.013254,1.847134e+09,1.191211e+09,0.011133,0.009431,0.003298,0.003581,0.003373,15.339794,0.00000,4.446988e-16,3.557590e-15
min,0.057494,7.619289e+09,1.421309e+08,0.059754,0.060356,0.069436,-0.005381,-0.005076,10.454056,3.79153,2.760540e+00,-1.620527e+01
25%,0.066806,9.204434e+09,3.108353e+08,0.066900,0.068265,0.074461,-0.002207,-0.002121,36.763874,3.79153,2.760540e+00,-1.620527e+01
50%,0.074670,1.034842e+10,4.663171e+08,0.075035,0.076263,0.078603,-0.000731,-0.000658,46.854450,3.79153,2.760540e+00,-1.620527e+01
75%,0.083685,1.156438e+10,7.464369e+08,0.084279,0.084088,0.080024,0.001310,0.001486,58.185360,3.79153,2.760540e+00,-1.620527e+01
max,0.142498,1.947676e+10,1.393532e+10,0.111347,0.095529,0.081554,0.018154,0.016247,100.000000,3.79153,2.760540e+00,-1.620527e+01


In [196]:
df_eos.describe()

,price,market_cap,total_volume,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
count,365.000000,3.650000e+02,3.650000e+02,365.000000,365.000000,365.000000,365.000000,357.000000,364.000000,365.000000,365.000000,365.000000
mean,1.029776,1.103692e+09,1.722120e+08,1.038086,1.048387,1.134309,-0.010300,-0.010667,47.457483,-7.078337,-19.501848,-34.738178
std,0.233233,2.221471e+08,1.530465e+08,0.220174,0.207989,0.121585,0.039260,0.037734,15.559575,0.000000,0.000000,0.000000
min,0.627266,6.962897e+08,3.932477e+07,0.681451,0.729087,0.892277,-0.093069,-0.085862,16.054397,-7.078337,-19.501848,-34.738178
25%,0.881034,9.613568e+08,9.157729e+07,0.889245,0.913241,1.086518,-0.040308,-0.041127,37.091817,-7.078337,-19.501848,-34.738178
50%,1.031858,1.100674e+09,1.230568e+08,1.053475,1.048952,1.107655,-0.007810,-0.007743,47.517744,-7.078337,-19.501848,-34.738178
75%,1.157284,1.219084e+09,1.980580e+08,1.148999,1.151159,1.238458,0.017588,0.017244,58.013319,-7.078337,-19.501848,-34.738178
max,1.842758,1.857778e+09,1.397465e+09,1.610842,1.547739,1.350484,0.123235,0.093836,100.000000,-7.078337,-19.501848,-34.738178


6 - Concatención de data frames

In [197]:
df_coins = pd.concat([df_solana,df_bitcoin,df_tether,df_usd_coin,df_ethereum,df_cardano,df_avalanche,df_bnb,df_dogecoin,df_eos])

In [198]:
df_coins.shape

(3650, 14)

In [199]:
df_coins.head()

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180
0,43.902581,1.532266e+10,1.018587e+09,15-08-2022,Solana,43.902581,43.902581,43.902581,0.000000,NaN,NaN,-8.05682,21.567564,5.268193
1,43.252084,1.506556e+10,1.075538e+09,16-08-2022,Solana,43.802505,43.854396,43.896109,-0.051892,NaN,0.0,-8.05682,21.567564,5.268193
2,40.556286,1.416116e+10,1.249878e+09,17-08-2022,Solana,43.303086,43.610092,43.862877,-0.307005,NaN,0.0,-8.05682,21.567564,5.268193
3,39.618895,1.386326e+10,8.249718e+08,18-08-2022,Solana,42.736288,43.314448,43.820648,-0.578160,NaN,0.0,-8.05682,21.567564,5.268193
4,35.989795,1.253047e+10,1.786764e+09,19-08-2022,Solana,41.698366,42.771881,43.742729,-1.073515,NaN,0.0,-8.05682,21.567564,5.268193


In [200]:
df_coins.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3650 entries, 0 to 364
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         3650 non-null   float64
 1   market_cap    3650 non-null   float64
 2   total_volume  3650 non-null   float64
 3   date          3650 non-null   object 
 4   nombre        3650 non-null   object 
 5   EMA12         3650 non-null   float64
 6   EMA26         3650 non-null   float64
 7   EMA200        3650 non-null   float64
 8   MACD          3650 non-null   float64
 9   signal_line   3570 non-null   float64
 10  rsi           3640 non-null   float64
 11  ROI30         3650 non-null   float64
 12  ROI90         3650 non-null   float64
 13  ROI180        3650 non-null   float64
dtypes: float64(12), object(2)
memory usage: 427.7+ KB


In [201]:
df_coins.isna().sum()

price            0
market_cap       0
total_volume     0
date             0
nombre           0
EMA12            0
EMA26            0
EMA200           0
MACD             0
signal_line     80
rsi             10
ROI30            0
ROI90            0
ROI180           0
dtype: int64

In [202]:
df_coins['Histogram'] = df_coins['MACD'] - df_coins['signal_line'] #Creación de columna histograma para grafico MACD 

In [203]:
df_coins.head()

,price,market_cap,total_volume,date,nombre,EMA12,EMA26,EMA200,MACD,signal_line,rsi,ROI30,ROI90,ROI180,Histogram
0,43.902581,1.532266e+10,1.018587e+09,15-08-2022,Solana,43.902581,43.902581,43.902581,0.000000,NaN,NaN,-8.05682,21.567564,5.268193,NaN
1,43.252084,1.506556e+10,1.075538e+09,16-08-2022,Solana,43.802505,43.854396,43.896109,-0.051892,NaN,0.0,-8.05682,21.567564,5.268193,NaN
2,40.556286,1.416116e+10,1.249878e+09,17-08-2022,Solana,43.303086,43.610092,43.862877,-0.307005,NaN,0.0,-8.05682,21.567564,5.268193,NaN
3,39.618895,1.386326e+10,8.249718e+08,18-08-2022,Solana,42.736288,43.314448,43.820648,-0.578160,NaN,0.0,-8.05682,21.567564,5.268193,NaN
4,35.989795,1.253047e+10,1.786764e+09,19-08-2022,Solana,41.698366,42.771881,43.742729,-1.073515,NaN,0.0,-8.05682,21.567564,5.268193,NaN


In [204]:
df_coins.isna().sum()

price            0
market_cap       0
total_volume     0
date             0
nombre           0
EMA12            0
EMA26            0
EMA200           0
MACD             0
signal_line     80
rsi             10
ROI30            0
ROI90            0
ROI180           0
Histogram       80
dtype: int64

In [205]:
df_coins.fillna(0, inplace=True)

In [206]:
# Guardar el DataFrame en un archivo CSV
df_coins.to_csv('Coins.csv', index=False)  # El parámetro index=False evita guardar el índice del DataFrame